# joining tables
a key concept for tidy data is the separation of data into tables, such that only relevant data is stored in each table. this reduces data size and increases readability. 

however, what data are relevant together can change based on the use case, so we often need to bring together into a single table information from two or more tables. the `join` operations that do this are the topic of today's session.

## today's exercise
consider the new york rodent inspection dataset from last time. we have already discussed how to read in that data, how to clean/manipulate the datetime information in it, and how to use group by and aggregation to calculate things like the monthly number of inspections. 

one might suspect that the weather plays a role in how many rodent inspections occur in a given day/week/month. to explore that hypothesis we must join the data on daily new york rodent inspection counts with data on the daily weather in new york. in the `data` folder there is a data file containing the daily weather summary for new york. read it in, count the number of inspections per day and join the result with the daily weather (precipitation (`PRCP` and `SNOW`) and temperature (`TMAX`, `TMIN`, and `TAVG`). then calculate the average number of daily inspections for days when the precipitation is nonzero, and the average number of daily inspaections for days when the precipitation is zero.

## note on image credits
this notebook contains links to animated gifs which i copied from here: https://github.com/gadenbuie/tidyexplain

# the art of the join
- in the following, when we talk of tables we mean a `pandas` dataframe.
- when we talk about joining tables, order matters. that is, joining table `x` with table `y` is in general not the same as joining `y` to `x`.
- in order to be able to join two tables into a single table, they must have at least one column in common, a column that is the key to the join and lets us know which row of one table to match against the other. 
- the resulting joined table retains some subset of the rows and columns of the two tables. 
- we will be using the `pd.merge()` function:
```
pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)
```
consider two tables, `x` and `y` defined as dataframes below (for comparison to this week's exercise, imagine  `x` has information on the count of inspections per day, and table `y` has the daily weather information):

In [1]:
import pandas as pd
x = pd.DataFrame([{'a':1, 'b':'x1'}, {'a':2, 'b':'x2'}, {'a':3, 'b':'x3'}], index=[1,2,3])
y = pd.DataFrame([{'a':1, 'c':'y1'}, {'a':2, 'c':'y2'}, {'a':4, 'c':'y4'}], index=[11,12,13])

In [2]:
print(x)

   a   b
1  1  x1
2  2  x2
3  3  x3


In [3]:
print(y)

    a   c
11  1  y1
12  2  y2
13  4  y4


note that `x` and `y` have one column in common: column `a`. each dataframe also has another column unique to itself. 
it is a good idea to pause here and think about what you expect the resulting joined table to look like? in fact there are several different ways to join these tables with different results. 

let's start with the simplest.

## full outer join
a *full outer join* of `x` and `y` is a table containing all the rows of `x` and all the rows of `y`, matched up on columns in common. wherever one table has a gap (lacks a value of the key), a `NaN` is inserted in its columns. 
![full join](images/full-join.gif)
this is the kind of join we would use when we are concerned about not dropping any of the data. 

here is how we can execute this kind of join:

In [4]:
outer_join = pd.merge(x,y,how='outer')
print(outer_join)

   a    b    c
0  1   x1   y1
1  2   x2   y2
2  3   x3  NaN
3  4  NaN   y4


note that the index is not being used.

## inner join
an *inner join* of `x` and `y` is a table containing only the rows of `x` and only the rows of `y` that have a match in the key column.
![inner join](images/inner-join.gif)
this is the kind of join we would often use when we want to investigate (model or plot) the relantionship between columns `b` and `c` and cannot use rows where either is missing.

In [5]:
inner_join = pd.merge(x, y, how='inner', on='a')
print(inner_join)

   a   b   c
0  1  x1  y1
1  2  x2  y2


## left join
a *left join* of `x` and `y` is a table containing all the rows of `x` and only the rows of `y` which have matching values in the key column(s).
![left join](images/left-join.gif)
this is the most frequently used type of join. we use this kind of join when `x` is our main data table that we do not want to drop rows from, but we are augmenting it with some other external data, `y`.

In [6]:
left_join = pd.merge(x,y,how='left', on='a')
print(left_join)

   a   b    c
0  1  x1   y1
1  2  x2   y2
2  3  x3  NaN


## right join
a *right join* of `x` and `y` is a table containing all the rows of `y` and only the rows of `x` which have matching values in the key column(s).
![right join](images/right-join.gif)
right join of `x` and `y` is exactly like a left join of `y` and `x`. what matters is which table is the one whose data we are not willing to drop in case of missing key values

In [7]:
left_join = pd.merge(x,y,how='right')
print(left_join)

   a    b   c
0  1   x1  y1
1  2   x2  y2
2  4  NaN  y4


## specifying columns
in case there are no identically named columns between the two tables, we must specify the column in each table

In [8]:
x = pd.DataFrame([{'x_a':1, 'x_b':'x1'}, {'x_a':2, 'x_b':'x2'}, {'x_a':3, 'x_b':'x3'}], index=[1,2,3])
print(x)
y = pd.DataFrame([{'y_a':1, 'y_c':'y1'}, {'y_a':2, 'y_c':'y2'}, {'y_a':4, 'y_c':'y4'}], index=[1,2,3])
print(y)

   x_a x_b
1    1  x1
2    2  x2
3    3  x3
   y_a y_c
1    1  y1
2    2  y2
3    4  y4


In [9]:
inner_join = pd.merge(x, y, how='inner', left_on='x_a', right_on='y_a')
print(inner_join)

   x_a x_b  y_a y_c
0    1  x1    1  y1
1    2  x2    2  y2


In [10]:
left_join = pd.merge(x, y, how='left', left_on='x_a', right_on='y_a').drop(['y_a'], axis=1)
print(left_join)

   x_a x_b  y_c
0    1  x1   y1
1    2  x2   y2
2    3  x3  NaN


# filtered joins

## left semi join
a *left semi join* of `x` and `y` is a table which retains only the columns of `x` and only the rows where `x` and `y` have matching keys. i
![semi join](images/semi-join.gif)
(a rarely-seen *right semi join* works similarly) in effect, the rows of `x` get filtered by the intersection of the keys in `x` and `y`. we use a semi join to trim a dataframe based on another dataframe or list.

In [11]:
boolean_mask = x['a'].isin(y['a'])
print(boolean_mask)
semi_join = x.loc[boolean_mask]
print(semi_join)

KeyError: 'a'

## intersect
when `x` and `y` have some number of columns in common, an *intersect join* of `x` and `y` is a table with the common columns, containing only the rows of `x` and `y` which have matching values in those columns, dropping the rows that only occur in each tables.
![intersect join](images/intersect.gif)

In [ ]:
x = pd.DataFrame([{'a':1, 'b':'x1'}, {'a':2, 'b':'x2'}, {'a':3, 'b':'x3'}])
y = pd.DataFrame([{'a':1, 'b':'x1', 'c':'y1'}, {'a':2, 'b':'x4', 'c':'y2'}, {'a':4, 'b':'x1', 'c':'y4'}])
print(x)
print(y)

In [ ]:
boolean_mask = x['a'].isin(y['a']) & x['b'].isin(y['b'])
#print(boolean_mask)
intersect = y.loc[boolean_mask].drop(['c'], axis=1)
print(intersect)

this gets simpler when the dataframes have the same columns:

In [ ]:
x = pd.DataFrame([{'col1':1, 'col2':'a'}, {'col1':1, 'col2':'b'}, {'col1':2, 'col2':'a'}])
y = pd.DataFrame([{'col1':1, 'col2':'a'}, {'col1':2, 'col2':'b'}])
print(x)
print(y)


In [ ]:
intersect_join = pd.merge(x,y,how='inner')
print(intersect_join)

# Exercise solution

In [12]:
import pandas as pd
from datetime import datetime

In [13]:
filename_csv = 'data/NY_rodent_inspections_sample.csv' # for full on run
rodent_df = pd.read_csv(filename_csv) 
# inspect the data frame
# rodent_df.shape
rodent_df.head

<bound method NDFrame.head of      INSPECTION_TYPE  JOB_TICKET_OR_WORK_ORDER_ID    JOB_ID  JOB_PROGRESS  \
0               BAIT                            1   PO12965             3   
1               BAIT                            2   PO12966             3   
2               BAIT                           30   PO16966             3   
3               BAIT                           31   PO13665             3   
4               BAIT                           38   PO11291             3   
5               BAIT                           39   PO12483             3   
6               BAIT                           41   PO25356             3   
7               BAIT                           46   PO16381             3   
8               BAIT                           48   PO11801             3   
9               BAIT                           50   PO11803             3   
10              BAIT                           51   PO11817             3   
11              BAIT                          

In [14]:
weather_filename = "data/ny_weather_data_daily.csv"
weather_df = pd.read_csv(weather_filename) 

In [15]:
format_descriptor = '%m/%d/%Y %I:%M:%S %p'
rodent_df['inspection_datetime']   = rodent_df.apply(lambda row: datetime.strptime(row['INSPECTION_DATE'], format_descriptor), axis=1)
rodent_df['inspection_day']      = rodent_df.apply(lambda row: row['inspection_datetime'].strftime("%Y-%m-%d"),   axis=1)

In [16]:
rodent_df['inspection_day']

0       2009-10-14
1       2009-10-14
2       2009-11-09
3       2009-11-09
4       2009-11-10
5       2009-11-10
6       2009-11-10
7       2009-11-16
8       2009-11-16
9       2009-11-16
10      2009-11-16
11      2009-11-16
12      2009-11-16
13      2009-11-17
14      2009-11-17
15      2009-11-17
16      2009-11-17
17      2009-11-17
18      2009-11-18
19      2009-11-18
20      2009-11-20
21      2009-11-20
22      2009-11-20
23      2009-11-20
24      2009-11-20
25      2009-11-20
26      2009-11-20
27      2009-11-23
28      2009-11-23
29      2009-11-23
           ...    
9969    2010-07-14
9970    2010-07-14
9971    2010-07-14
9972    2010-07-06
9973    2010-07-14
9974    2010-07-14
9975    2010-07-14
9976    2010-07-14
9977    2010-07-14
9978    2010-07-14
9979    2010-07-14
9980    2010-07-15
9981    2010-07-15
9982    2010-07-15
9983    2010-07-15
9984    2010-07-15
9985    2010-07-15
9986    2010-07-15
9987    2010-07-15
9988    2010-07-15
9989    2010-07-15
9990    2010

In [17]:
weather_format_descriptor = '%Y-%m-%d'

weather_df["day"] = weather_df.apply(lambda row: datetime.strptime(row['DATE'], weather_format_descriptor), axis=1)

In [18]:
weather_df["day"].describe()

count                    3591
unique                   3591
top       2015-10-18 00:00:00
freq                        1
first     2009-01-01 00:00:00
last      2018-10-31 00:00:00
Name: day, dtype: object

In [19]:
inspection_day_counts = rodent_df.groupby('inspection_day').agg({'inspection_day':'count'})
inspection_day_counts = inspection_day_counts.rename(columns={"inspection_day" : "day_counts"})
# inspection_day_counts.reindex()
inspection_day_counts["date"] = inspection_day_counts.index
inspection_day_counts.head()

,day_counts,date
inspection_day,,
2009-01-16,1,2009-01-16
2009-10-14,2,2009-10-14
2009-11-09,2,2009-11-09
2009-11-10,3,2009-11-10
2009-11-16,6,2009-11-16


In [20]:
inspection_day_counts["day"] = inspection_day_counts.index

In [21]:
inspection_day_counts["date"] = pd.to_datetime(inspection_day_counts["date"])

In [22]:
inspection_day_counts.head()

,day_counts,date,day
inspection_day,,,
2009-01-16,1,2009-01-16,2009-01-16
2009-10-14,2,2009-10-14,2009-10-14
2009-11-09,2,2009-11-09,2009-11-09
2009-11-10,3,2009-11-10,2009-11-10
2009-11-16,6,2009-11-16,2009-11-16


In [42]:
inspection_day_counts

,day_counts,date,day
inspection_day,,,
2009-01-16,1,2009-01-16,2009-01-16
2009-10-14,2,2009-10-14,2009-10-14
2009-11-09,2,2009-11-09,2009-11-09
2009-11-10,3,2009-11-10,2009-11-10
2009-11-16,6,2009-11-16,2009-11-16
2009-11-17,5,2009-11-17,2009-11-17
2009-11-18,2,2009-11-18,2009-11-18
2009-11-20,7,2009-11-20,2009-11-20
2009-11-23,3,2009-11-23,2009-11-23


In [24]:
days_and_weather = pd.merge(inspection_day_counts,weather_df,how='left', left_on='date', right_on='day')
days_and_weather

,day_counts,date,day_x,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,...,WT13,WT14,WT15,WT16,WT17,WT18,WT19,WT21,WT22,day_y
0,1,2009-01-16,2009-01-16,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,2009-01-16,6.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-01-16
1,2,2009-10-14,2009-10-14,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,2009-10-14,3.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-10-14
2,2,2009-11-09,2009-11-09,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,2009-11-09,2.8,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-11-09
3,3,2009-11-10,2009-11-10,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,2009-11-10,2.9,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-11-10
4,6,2009-11-16,2009-11-16,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,2009-11-16,4.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-11-16
5,5,2009-11-17,2009-11-17,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,2009-11-17,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-11-17
6,2,2009-11-18,2009-11-18,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,2009-11-18,3.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-11-18
7,7,2009-11-20,2009-11-20,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,2009-11-20,6.2,...,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2009-11-20
8,3,2009-11-23,2009-11-23,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,2009-11-23,6.2,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2009-11-23
9,4,2009-11-24,2009-11-24,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.6386,-73.7622,3.4,2009-11-24,4.3,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2009-11-24


In [25]:
days_and_weather.columns

Index(['day_counts', 'date', 'day_x', 'STATION', 'NAME', 'LATITUDE',
       'LONGITUDE', 'ELEVATION', 'DATE', 'AWND', 'FMTM', 'PGTM', 'PRCP',
       'SNOW', 'SNWD', 'TAVG', 'TMAX', 'TMIN', 'WDF2', 'WDF5', 'WSF2', 'WSF5',
       'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT07', 'WT08', 'WT09',
       'WT11', 'WT13', 'WT14', 'WT15', 'WT16', 'WT17', 'WT18', 'WT19', 'WT21',
       'WT22', 'day_y'],
      dtype='object')

In [26]:
days_and_weather["total_precipitation"] = days_and_weather["SNOW"] + days_and_weather["PRCP"]

In [27]:
import numpy
days_and_weather["precipitation_bool"] = numpy.where(days_and_weather["total_precipitation"]>0, 1,0)

In [28]:
precip_summary = days_and_weather.groupby('precipitation_bool').agg({'precipitation_bool':'count'})
precip_summary

,precipitation_bool
precipitation_bool,
0,124
1,49


In [32]:
# days_and_weather

In [33]:
# inspection_day_counts.reset_index()

In [38]:
simple_days_and_weather = days_and_weather[["precipitation_bool", "date"]]


In [43]:
inspection_day_counts.keys()

Index(['day_counts', 'date', 'day'], dtype='object')

In [45]:
simple_days_and_weather.keys()

Index(['precipitation_bool', 'date'], dtype='object')

In [46]:
inpections_and_weather =pd.merge(inspection_day_counts,simple_days_and_weather,how='left', left_on='date', right_on='date')

In [48]:
inpections_and_weather.groupby()

,day_counts,precipitation_bool
count,173.000000,173.000000
mean,57.797688,0.283237
std,37.747318,0.451878
min,1.000000,0.000000
25%,29.000000,0.000000
50%,55.000000,0.000000
75%,88.000000,1.000000
max,140.000000,1.000000


In [68]:
final_answer = inpections_and_weather.groupby("precipitation_bool")["day_counts"].mean()

In [69]:
final_answer

precipitation_bool
0    54.693548
1    65.653061
Name: day_counts, dtype: float64